In [ ]:
# Import Built-Ins
import datetime
from copy import deepcopy
# Import Third-Party
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Import Homebrew
from utilities.functions import timeit, preprocess_order_file, create_limit_order_book
from utilities.limit_order_book_og import LimitOrderBookOg, OrderOg
from utilities.limit_order_book import LimitOrderBook, Order


pd.set_option('display.max_columns', None)
sns.set()
timestamp = datetime.datetime(year=2017, month=1, day=2, hour=11, minute=30)

# Data formating

In [ ]:
path = '/Users/australien/Documents/IESEG/Master 2/Thesis/BEDOFIH 2017/Data/01/20170102/AA0000000000-AN8068571086/AN8068571086/VHOX_AN8068571086_20170102.csv'

In [ ]:
data_test = preprocess_order_file(path=path, save=False)
data_test.head()

In [ ]:
data_test.to_csv("/Users/australien/Downloads/VHOX_orders_test_processed.csv")

# Data discovery

In [ ]:
data = data_test.copy()
data.info()

In [ ]:
data[data.o_id_fd == 17568959537]

In [ ]:
data[data['o_state']=='4'].head()

In [ ]:
data[data['o_type']=='1'].head()

# Limit order book (All data)
## Creation

In [ ]:
"""
timestamp_test = data.loc[74851, 'time_validity']
timestamp_issue = data[(data['o_id_fd'] == 5491124529) & (data['o_id_cha'] == 6)].iloc[0,:].time_submission
timestamp_issue
lob = create_limit_order_book(data, timestamp_issue)
"""

lob = create_limit_order_book(orders=data[:131], limit_timestamp=timestamp)

In [ ]:
lob.levels(50)['bids']

In [ ]:
lob.levels(5)['asks']

In [ ]:
lob._orders

In [ ]:

#lob = create_limit_order_book(data_issue, timestamp)
lob = create_limit_order_book(data, timestamp)
lob._price_levels


## Exploration of capabilities

In [ ]:
print(lob.best_ask.orders.head)

In [ ]:
lob._orders[5491124525]

In [ ]:
row = data.iloc[74852, :]
if row.o_type == '2':
    time_validity = row.time_validity
    id = row.o_id_fd
    char_id = row.o_id_cha
    is_bid = True if row.o_bs == 'B' else False
    ini_size = row.o_q_ini
    neg_size = row.o_q_neg
    rem_size = row.o_q_rem
    price = row.o_price
    state = row.o_state
    o_type = row.o_type
    account = row.o_account
    hft_flag = row.o_member
    time_submission = row.time_submission
    order = Order(uid=id, char_id=char_id, is_bid=is_bid, ini_size=ini_size, neg_size=neg_size, 
                    rem_size=rem_size, price=price, state=state, type=o_type, account=account, 
                    hft_flag=hft_flag, sub_timestamp=time_submission, val_timestamp=time_validity)
order

In [ ]:
lob.process(order)

In [ ]:
lob._price_levels

In [ ]:
lob.top_level

In [ ]:
print(lob.best_ask)
print(lob.best_bid)

In [ ]:
lob_copy = deepcopy(lob)

# Test of order update (One order)

In [ ]:
# Find id or order with 337 updates
id = data[data['o_id_cha'] == 337]['o_id_fd'].iloc[0]
data_test = data[data['o_id_fd'] == id].sort_values(by='o_id_cha')
#data_test.to_excel('/Users/australien/Downloads/one_order_updated_2.xlsx')
data_test.tail()

In [ ]:

lob_test = create_limit_order_book(data_test, timestamp)
lob_test._orders

In [ ]:
lob.levels(2)

In [20]:
import pandas as pd


path = '/Volumes/Extreme ssd/trades/CH0012214059/VHD_CH0012214059_20170102.csv'

data = pd.read_csv(path)
data['t_dtm_neg'] = pd.to_datetime(data['t_dtm_neg'])
date = data.iloc[0].t_dtm_neg.time()
date

datetime.time(9, 0, 13, 10191)

In [22]:
import datetime
CLOSING_AUCTION_CUTOFF = datetime.time(hour=18, minute=35, second=0)

df = data[data.t_dtm_neg.dt.time > CLOSING_AUCTION_CUTOFF]
df.empty

True